### 多继承mixin的设计实践

Mixin 是一种在面向对象编程中常用的设计模式（**组合模式**），用于在类之间共享方法而不使用传统的多重继承。Mixin 类是一种包含一组方法但不定义构造函数（`__init__`）的类，通常作为其他类的超类，通过多重继承来注入功能。

#### Mixin 的特点：
- **独立性**：Mixin 类通常只包含一组相关方法（**功能单一**），没有状态信息（实例变量），并且不应该被单独实例化。它也**不与其他的基类强耦合**，其他的基类不继承Mixin类也可以实例化成功。
- **代码重用**：Mixin 类可以被多个类共同继承，作为一种接口它**不会与其他类强产生耦合**，从而实现方法的复用，避免代码冗余。
- **功能增强**：Mixin 类通过多继承与其他类组合，为这些类提供额外的功能，增强其行为。

#### Mixin 的优势：
- **灵活性**：将功能划分为独立的 Mixin 类，使类的设计更具灵活性，易于维护和扩展。
- **避免深层次继承**：通过 Mixin，可以避免深层次的继承结构带来的复杂性和潜在问题。
- **模块化**：将功能模块化，降低耦合度，提高代码的可读性和可维护性。  

举例：Django自带的Admin类，如果需要对其添加导出文件时自定义实现命名规则的特性  
可以新建一个Mixin类重写`get_export_filename`方法，并将其通过多继承的方式引入需要实现该特性的Admin  
> 以下的代码不需要运行（也没法运行），看Mixin的具体实现和用法

In [ ]:
from django.contrib import admin
from import_export.admin import ExportMixin
from import_export.admin import ImportExportModelAdmin
from urllib.parse import quote
from datetime import datetime

In [ ]:
# 用于自定义导出文件名称的Mixin Class
class ExportFileNameMixin(ExportMixin):
    # 重写ExportMixin的get_export_filename方法
    def get_export_filename(self, request, queryset, file_format):
        # 获取模型名称
        filename = self.model._meta.verbose_name
        # 返回自定义的名称
        return "%s_%s.%s" % (
            quote(filename),  # 文件名需要编码为ascii形式
            datetime.now().strftime("%Y-%m-%d"),
            file_format.get_extension()
        )


# 虚构的CustomModelAdmin Class
@admin.register(CustomModel)
class CustomModelAdmin(ExportFileNameMixin, ImportExportModelAdmin):
    pass
    """
    ... 自定义若干功能
    """


可以看到`CustomModelAdmin`同时继承Mixin和需要用到的导出模型类  
此时设计的Mixin重写的`get_export_filename`方法就会在被继承的`CustomModelAdmin`中生效  
并且CustomModelAdmin与继承的Mixin之间没有强关联，去掉也不影响其正常运行   

注意，多继承Class的顺序是**从左往右**计算的，所以如果后一个类中有会影响前一个类重写的内容，需要将自定义的Mixin放在受影响的类顺序之前